### Get Sentence feature from a pretrained model

In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from util import load_directory_data,load_dataset,download_and_load_datasets,gpusession
from mlp_cls_class import mlp_cls_class
print ("Packages loaded.")

Packages loaded.


### Get feature

In [2]:
dataset_name = 'movie_review'

In [3]:
savename = 'data/' + dataset_name + '.npz'
FORCE_RELOAD = True 
if os.path.isfile(savename) & (~FORCE_RELOAD): # load if the file exists,
    # Load 
    l = np.load(savename)    
    x_train = l['x_train']
    x_test = l['x_test']
    t_train = l['t_train']
    t_test = l['t_test']
    print ("[%s] Loaded. Size is [%.4f]MB" % (savename,os.path.getsize(savename)/1000./1000.))
else: # otherwise, make the data
    # Download dataset
    tf.logging.set_verbosity(tf.logging.ERROR) # Reduce logging output.
    train_df, test_df = download_and_load_datasets()
    # Check dataset
    for i in range(2):
        print (i,':',train_df['sentence'][i],'\n')
    for i in range(2):
        print (i,':',train_df['polarity'][i])
    embed_module = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
    embed_train = embed_module(tf.reshape(train_df["sentence"], shape=[-1]))
    embed_test = embed_module(tf.reshape(test_df["sentence"], shape=[-1]))
    with tf.train.MonitoredTrainingSession(is_chief=True) as sess:
        x_train = sess.run(embed_train)
        x_test = sess.run(embed_test)
    n_train,n_test = np.shape(x_train)[0],np.shape(x_test)[0]
    t_train,t_test = np.zeros((n_train,2)),np.zeros((n_test,2))
    for i in range(n_train):
        t_train[i,train_df['polarity'][i]] = 1
    for i in range(n_test):
        t_test[i,test_df['polarity'][i]] = 1    
    print("Shapes of 'x_train' and 'x_test' are %s and %s."%
          (x_train.shape,x_test.shape)) # (result: (1, 128))    
    print("Shapes of 't_train' and 't_test' are %s and %s."%
          (t_train.shape,t_test.shape)) # (result: (1, 128))    
    # Save 
    np.savez(savename,x_train=x_train,x_test=x_test,t_train=t_train,t_test=t_test)
    print ("[%s] Saved. Size is [%.4f]MB" % (savename,os.path.getsize(savename)/1000./1000.))

0 : "An album of songs so old everyone thinks they're new." This film has the elusive combination of pace and mood that set some films apart from the opening moments. And why not? Towering talent from Dame Judith Dench as a widow who plays saxaphone with a street musician to help him get the songs right, to Olympia Dukakis as the merry widow living in a Scottish castle on the alimony of her many marriages, to Ian Holm as the drummer who loved all the members of a World War II all girl (more or less) swing band. But wait, there's more. Add in Leslie Caron on bass, and the incomparable Clio Laine on lead vocal, at last, and the Blonde Bombshells are the hottest band in England since the Beatles. Well, OK, not really, but this movie is a winner.<br /><br />Elizabeth (Dench) spends the whole film trying to reunite the Blonde Bombshells to play at her granddaughter's school dance. And before you roll your eyes, imagine how difficult and courageous it would be for a bunch of sexegenarian wom

### Run

In [4]:
tf.reset_default_graph(); tf.set_random_seed(0)
CNN = mlp_cls_class(_name='mlp_cls_sentiment',_x_dim=128,_t_dim=2,_h_dims=[128,64],
                _actv=tf.nn.relu,_bn=slim.batch_norm,_l2_reg_coef=1e-5,_momentum=0.5,
                _USE_SGD=False,_USE_MIXUP=False,_GPU_ID=0,_VERBOSE=True)
sess = gpusession();sess.run(tf.global_variables_initializer()) 
CNN.train(sess,_x_train=x_train,_t_train=t_train,_x_test=x_test,_t_test=t_test,
              _max_epoch=10,_batch_size=128,_lr=1e-3,_kp=0.9,
              _LR_SCHEDULE=False,_PRINT_EVERY=10,_VERBOSE_TRAIN=True)

==== Global Variables ====
 [00] Name:[mlp_cls_sentiment/lin0/weights:0] Shape:[[128, 128]]
 [01] Name:[mlp_cls_sentiment/lin0/BatchNorm/beta:0] Shape:[[128]]
 [02] Name:[mlp_cls_sentiment/lin0/BatchNorm/moving_mean:0] Shape:[[128]]
 [03] Name:[mlp_cls_sentiment/lin0/BatchNorm/moving_variance:0] Shape:[[128]]
 [04] Name:[mlp_cls_sentiment/lin1/weights:0] Shape:[[128, 64]]
 [05] Name:[mlp_cls_sentiment/lin1/BatchNorm/beta:0] Shape:[[64]]
 [06] Name:[mlp_cls_sentiment/lin1/BatchNorm/moving_mean:0] Shape:[[64]]
 [07] Name:[mlp_cls_sentiment/lin1/BatchNorm/moving_variance:0] Shape:[[64]]
 [08] Name:[mlp_cls_sentiment/out/weights:0] Shape:[[64, 2]]
 [09] Name:[mlp_cls_sentiment/out/biases:0] Shape:[[2]]
 [10] Name:[mlp_cls_sentiment/lin0/weights/Adam:0] Shape:[[128, 128]]
 [11] Name:[mlp_cls_sentiment/lin0/weights/Adam_1:0] Shape:[[128, 128]]
 [12] Name:[mlp_cls_sentiment/lin0/BatchNorm/beta/Adam:0] Shape:[[128]]
 [13] Name:[mlp_cls_sentiment/lin0/BatchNorm/beta/Adam_1:0] Shape:[[128]]
 [14